In [1]:
import timeit
from io import StringIO
import numba
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import urllib.request
import numpy as np
from datetime import datetime
# print message after packages imported successfully
from numba import jit
from past.builtins import raw_input
from scipy import sparse
print("import of packages successful")
##import of packages successful

import of packages successful


In [2]:
path = raw_input("Please enter the path to nf_prize_dataset , for exemple /home/mohamedali/Desktop/nf_prize_dataset/download") 
os.chdir(path)

In [3]:
#/home/mohamedali/Desktop/Matrix_factorization/nf_prize_dataset/download
#Data Path


# Methodology

## 1. Data Compilation

### 1.1. Generation of the rating dataset

* Write Python function **rating_compiler** to compile from the training_set.tar, the $17770$ files and store the result into one data structure named $\bf{D}$;

* $\bf{D}$ can be a numpy matrix, csv file/pandas DataFrame, sparse matrix, hdf5 file...;



In [3]:
def rating_compiler(previous=None):
    for i in range(1, 3):
        file = "training_set" + "/mv_00" + '{0:05}'.format(i) + '.txt'
        dt=np.dtype({'names': ('CustomerID','Rating','Date'),'formats': ('int32', 'int8', 'datetime64[D]')})
        data = np.loadtxt(fname = file,skiprows=1, delimiter=',', dtype=dt)
        new_data = np.array(data.tolist())
        leng = np.shape(data)[0]
        movie_id = np.full((leng,1),i)
        new_data = np.append(new_data, movie_id, axis=1)
        print(i)
        if(i==1):
            previous = new_data
        else:
            previous = np.concatenate((previous, new_data))

    return(previous)

In [4]:
D = rating_compiler()

1
2


In [5]:
print(D)

[[1488844 3 datetime.date(2005, 9, 6) 1]
 [822109 5 datetime.date(2005, 5, 13) 1]
 [885013 4 datetime.date(2005, 10, 19) 1]
 ...
 [1824543 4 datetime.date(2005, 4, 20) 2]
 [1283204 3 datetime.date(2004, 12, 23) 2]
 [1272122 5 datetime.date(2005, 7, 25) 2]]


### 1.2. Generation of training and test datasets

* Extract from $\bf{D}$ the ratings corresponding to the users and movies described in the probe.txt and store the result into one data structure named $\bf{T}$;

In [ ]:
#R=D-T
#T=D.ratings==probe.txt

In [6]:
#Extract the character before :
def substring_after(s, delimiter):
    return s.partition(delimiter)[0]

In [7]:
def getting_t():
    the_array = np.array([])
    with open('t.txt') as f:
        customer_id = 0
        movie_ids = 0
        for line in f:
           # For Python3, use print(line)
           #Compare if it is a Movie or a Customer Id
           # #stoke the movie id
            if ':' in line:
                movie_ids = int(substring_after(line,":"))
                print("Movie id",movie_ids)

            elif ":" not in line:
                customer_id = int(line)
                print("Customer_id : ",customer_id)

                if the_array.size == 0:
                    the_array= np.select(((D[:,0]==customer_id) * (D[:,3]==movie_ids)),D).reshape((1,4))
                else:
                    the_array = np.concatenate((the_array,np.select(((D[:,0]==customer_id) * (D[:,3]==movie_ids)),D).reshape((1,4))))


            if 'str' in line:
                break
    print("Finish")
    return the_array

In [8]:
T = getting_t()

Movie id 1
Customer_id :  30878
Customer_id :  2647871
Customer_id :  1283744
Customer_id :  2488120
Customer_id :  317050
Customer_id :  1904905
Customer_id :  1989766
Customer_id :  14756
Customer_id :  1027056
Customer_id :  1149588
Customer_id :  1394012
Customer_id :  1406595
Customer_id :  2529547
Customer_id :  1682104
Customer_id :  2625019
Customer_id :  2603381
Customer_id :  1774623
Customer_id :  470861
Customer_id :  712610
Customer_id :  1772839
Customer_id :  1059319
Customer_id :  2380848
Customer_id :  548064
Finish


* Construct training dataset $\bf{R}$ as $\bf{D}$ from which we remove entries present in $\bf{T}$;

* $\bf{D}$, $\bf{T}$ and $\bf{R}$ must all have the same format (for example users as rows and films as columns);

In [9]:
D_pd = pd.DataFrame({'Customer_id': D[:, 0], 'Rating': D[:, 1], 'Date': D[:, 2], 'Movie_id': D[:, 3]})
T_pd = pd.DataFrame({'Customer_id': T[:, 0], 'Rating': T[:, 1], 'Date': T[:, 2], 'Movie_id': T[:, 3]})

In [10]:
df = pd.DataFrame({'userID':D[:, 0], 'itemID': D[:, 3], 'rating' : D[:, 1]})
tf = pd.DataFrame({'userID':T[:, 0], 'itemID': T[:, 3], 'rating' : T[:, 1]})

In [11]:
df1 = df
df2 = tf

In [12]:
#First Method
R_pd1= df1[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]

In [13]:
#Second Method
R_pd2 = T_pd.merge(D_pd, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']

In [14]:
data = 'string or data-table to pass'
%store data
del data

Stored 'data' (str)


In [13]:
%store df
%store tf
%store R_pd1


Stored 'df' (DataFrame)
Stored 'tf' (DataFrame)
Stored 'R_pd1' (DataFrame)


In [14]:
#Sparse Matrix
##Initializing the Data frame
dt=np.dtype({'names': ('MovieID','Rating','Date'),'formats': ('int16', 'int8', 'datetime64[D]')})
D = np.zeros((2649429,4), dtype=dt)
##Data frame size
print("%d Megabytes" % ((D.size * D.itemsize)/1000000))

116 Megabytes


In [15]:
#The D sparse Matrix
D_col = df.itemID.astype(dtype='int16')
D_row = df.userID.astype(dtype='int32')
D_data = df.rating.astype(dtype='int8')
D_sparse= sparse.csr_matrix((D_data, (D_row, D_col)))

In [16]:
#The T sparse Matrix
T_col = tf.itemID.astype(dtype='int16')
T_row = tf.userID.astype(dtype='int32')
T_data = tf.rating.astype(dtype='int8')
T_sparse= sparse.csr_matrix((T_data, (T_row, T_col)))

matrix([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 4]], dtype=int8)

In [ ]:
TestUISparseData.todense()

In [59]:
print(TestUISparseData[2059652,2])

4
